In [ ]:
!pwd

In [ ]:
!ls

In [ ]:
!ls ..

In [ ]:
!ls ../data

In [ ]:
!nvidia-smi

In [ ]:
!python -m torch.utils.collect_env 

In [ ]:
!nvcc --version

In [ ]:
!python -m bitsandbytes

In [ ]:
!pip install fastparquet

In [ ]:
!ls /mystuff/wikipedia

In [ ]:
import pandas as pd

df_a = pd.read_parquet("/mystuff/wikipedia/a.parquet")


In [ ]:
import os
import pandas as pd

# Define the folder path containing the .parquet files
folder_path = '/mystuff/wikipedia'

# List all files with .parquet extension
parquet_files = [f for f in os.listdir(folder_path) if f.endswith('.parquet') and "wiki_2023" not in f]

# Load each file into a DataFrame and store in a dictionary
dataframes = {}
for file in parquet_files:
    key = file.rsplit('.', 1)[0]  # Remove the .parquet extension to get the key
    dataframes[key] = pd.read_parquet(os.path.join(folder_path, file))

print(dataframes.keys())  # This will print all the keys (i.e., filenames without .parquet extension)


In [ ]:
df_a.columns

In [ ]:
print(df_a.iloc[0].text)

In [ ]:
print(df_a.iloc[0].text)

In [ ]:
import pandas as pd
import re

def extract_identifiers(text):
    # The regular expression looks for '==', followed by any number of characters 
    # that aren't '=', followed by another '=='. This captures the IDENTIFIER.
    return re.findall(r'==([^=]+)==', text)




In [ ]:
# Apply the function to the 'text' column and get the unique values
unique_identifiers = df_a['text'].apply(extract_identifiers).explode().unique()

print(unique_identifiers)

In [ ]:
import pandas as pd
import re

# Applying the function to extract identifiers for each document
df_a['identifiers'] = df_a['text'].apply(extract_identifiers)

# Finding the common identifiers across all documents
common_identifiers = set(df_a['identifiers'].iloc[0])
for ids in df_a['identifiers']:
    common_identifiers.intersection_update(ids)

print(common_identifiers)


In [ ]:
# Applying the function to extract identifiers for each document
all_identifiers = df_a['text'].apply(extract_identifiers).explode()

# Getting the count of each identifier and displaying the top 10
top_10_identifiers = all_identifiers.value_counts().head(20)
print(top_10_identifiers)

In [ ]:
df_a.shape

In [ ]:
from tqdm.notebook import tqdm

def extract_raw_identifiers(text):
    return re.findall(r'==([^=]+)==', text)

# Extracting all raw identifiers
print("extracting")
df_a['raw_identifiers'] = df_a['text'].apply(extract_raw_identifiers)
print("exploding")
df_exploded = df_a.explode('raw_identifiers')

print("unique")
# Grouping by stripped identifiers to gather variations
identifier_variations = (df_exploded.groupby(df_exploded['raw_identifiers'].str.strip())
                         ['raw_identifiers'].unique())

print("keeping only multiple variations")
# Filtering to keep only entries with multiple variations
identifier_variations = identifier_variations[identifier_variations.str.len() > 1]



In [ ]:
identifier_variations

In [ ]:
# Fetching example documents for each variation
example_docs = {}
for identifier, variations in tqdm(identifier_variations.items()):
    example_docs[identifier] = {}
    for variation in variations:
        example_text = df_exploded[df_exploded['raw_identifiers'] == variation]['text'].iloc[0]
        example_docs[identifier][variation] = example_text

for identifier, docs in example_docs.items():
    print(f"Identifier: {identifier}")
    for variation, doc in docs.items():
        print(f"  Variation: {variation}\n  Example Document: {doc}\n")
    print("-" * 50)
    break


In [ ]:
# Extracting identifiers directly from text without normalization
def extract_raw_identifiers(text):
    return re.findall(r'==([^=]+)==', text)

df_a['raw_identifiers'] = df_a['text'].apply(extract_raw_identifiers)
df_exploded = df_a.explode('raw_identifiers')

print("get value counts")
# Normalizing identifiers and counting their frequencies
identifier_counts = df_exploded['raw_identifiers'].str.strip().value_counts()

# Filter normalized identifiers with count over 1000
print("filter over 1000")
popular_identifiers = identifier_counts[identifier_counts > 1000].index


In [ ]:
from tqdm.notebook import tqdm

print("collect example docs")
# Gather variations for these identifiers and fetch example docs
example_docs = {}
for identifier in tqdm(popular_identifiers):
    variations = df_exploded[df_exploded['raw_identifiers'].str.strip() == identifier]['raw_identifiers'].unique()
    
    # Check for multiple variations
    if len(variations) > 1:
        example_docs[identifier] = {}
        for variation in variations:
            example_text = df_exploded[df_exploded['raw_identifiers'] == variation]['text'].iloc[0]
            example_docs[identifier][variation] = example_text


In [ ]:
print("print example docs")
for identifier, docs in tqdm(example_docs.items()):
    print(f"Identifier: {identifier}")
    for variation, doc in docs.items():
        print(f"  Variation: '{variation}'\n  Example Document: {doc}\n")
    print("-" * 50)
    break

In [ ]:

# Extracting identifiers directly from text without normalization
def extract_raw_identifiers(text):
    return re.findall(r'==([^=]+)==', text)

# Normalize and count identifiers
df_exploded['normalized_identifiers'] = df_exploded['raw_identifiers'].str.strip()
identifier_counts = df_exploded['normalized_identifiers'].value_counts()

# Only consider identifiers with more than 1000 occurrences
popular_identifiers = identifier_counts[identifier_counts > 1000].index

# Filter the mapping dictionary to consider only prevalent identifiers
variation_to_normalized = {
    var: var.strip()
    for var in df_exploded['raw_identifiers'].unique()
    if isinstance(var, str) and var.strip() in popular_identifiers
}


In [ ]:
len(variation_to_normalized)

In [ ]:
%%time
# Replace all variations with the normalized form using regex
pattern = re.compile('==' + '|'.join(map(re.escape, variation_to_normalized.keys())) + '==')
#def replace_with_normalized(text):
#    return pattern.sub(lambda m: '==' + variation_to_normalized[m.group(0)] + '==', text)
def replace_with_normalized(text):
    return pattern.sub(lambda m: '==' + variation_to_normalized.get(m.group(0), m.group(0).strip()) + '==', text)

df_a['normalized_text'] = df_a['text'].apply(replace_with_normalized)
#df_a['normalized_text'] = df_a['text'].progress_apply(replace_with_normalized)

print(df_a[['text', 'normalized_text']])


In [ ]:
tqdm.pandas()

# Extracting identifiers directly from text without normalization
def extract_raw_identifiers(text):
    return re.findall(r'==([^=]+)==', text)

def normalize_data(df_to_normalize):
    # Extracting all raw identifiers
    df_to_normalize['raw_identifiers'] = df_to_normalize['text'].progress_apply(extract_raw_identifiers)
    df_exploded = df_to_normalize.explode('raw_identifiers')
    
    # Normalize and count identifiers
    df_exploded['normalized_identifiers'] = df_exploded['raw_identifiers'].str.strip()
    identifier_counts = df_exploded['normalized_identifiers'].value_counts()
    
    # Only consider identifiers with more than 1000 occurrences
    popular_identifiers = identifier_counts[identifier_counts > 100].index
    
    # Filter the mapping dictionary to consider only prevalent identifiers
    variation_to_normalized = {
        var: var.strip()
        for var in df_exploded['raw_identifiers'].unique()
        if isinstance(var, str) and var.strip() in popular_identifiers
    }
    
    # Replace all variations with the normalized form using regex
    pattern = re.compile('|'.join(map(lambda x: '==' + re.escape(x) + '==', variation_to_normalized.keys())))
    def replace_with_normalized(text):
        return pattern.sub(lambda m: '==' + variation_to_normalized.get(m.group(0).strip('=='), m.group(0).strip('==')) + '==', text)
    
    df_to_normalize['normalized_text'] = df_to_normalize['text'].progress_apply(replace_with_normalized)
    
    #print(df_to_normalize[['text', 'normalized_text']])
    return df_to_normalize

In [ ]:
normalized_dataframes = {}
for key in dataframes:
    print(f"normalizing {key}")
    df_normalized = normalize_data(dataframes[key])
    normalized_dataframes[key] = df_normalized
    print(f"normalizing {key} done")

In [ ]:
normalized_dataframes["a"]

In [ ]:
folder_path = '/mystuff/wikipedia'
for key in normalized_dataframes:
    normalized_dataframes[key].to_parquet(f"{folder_path}/normalized_{key}.parquet")

In [ ]:
# Applying the function to extract identifiers for each document
all_identifiers = normalized_dataframes["a"]['normalized_text'].apply(extract_identifiers).explode()

# Getting the count of each identifier and displaying the top 10
top_10_identifiers = all_identifiers.value_counts().head(20)
print(top_10_identifiers)